In [1]:
import os 
os.chdir("C:\\Users\\Pieter-Jan\\Documents\\Work\\Candriam\\nlp\\ESG\\topic_sentiment_analysis")

# Libraries

In [2]:
# data manipulations
import pandas as pd
from sklearn.feature_extraction import text

# data manipulations
import pandas as pd
import numpy as np
import re
from scipy import sparse

# CMT
from contextualized_topic_models.models.ctm import CTM
from contextualized_topic_models.datasets.dataset import CTMDataset
from contextualized_topic_models.evaluation.measures import CoherenceNPMI, InvertedRBO, CoherenceWordEmbeddings
from contextualized_topic_models.utils.preprocessing import SimplePreprocessing

# lDA
from gensim.corpora.dictionary import Dictionary
from gensim.test.utils import common_texts
from gensim.models import LdaMulticore 
import pyLDAvis.gensim
from gensim import corpora, matutils, models, similarities
import pyLDAvis

# BerTopic
from bertopic import BERTopic

# other
import random

# own modules
import modules.preprocessing as preprocess

sorflow\core\framework\types_pb2.py:177: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueDescriptor(
C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\types_pb2.py:181: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueDescriptor(
C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\tensorflow\core\framework\types_pb2.py:185: DeprecationWarning: Call to deprecated create function EnumValueDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.EnumValueD

In [3]:
%reload_ext autoreload
%autoreload 2

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Goal

We are going to compare the performance of three **unsupervised** models for topic modelling on ESG documents.

1. Contextualized Topic Modelling (CTM): https://github.com/MilaNLProc/contextualized-topic-models
2. Latent Dirichlet Allocation (LDA): https://radimrehurek.com/gensim/models/ldamodel.html
3. BERTopic: https://github.com/MaartenGr/BERTopic

# Evaluation measures 

1. **Normalized Point-wise Mutual Information (NPMI) (Lau et al.,
2014)**

It measures how much the top-10 words of a topic are related to each other, considering the empirical frequency of the words computed on the
original corpus. τ is a symbolic metric and relies on co-occurrence.

2. **External Word Embeddings Topic Coherence**

As Ding et al. (2018) pointed out, though, topic
coherence computed on the same data is inherently
limited. Coherence computed on an external corpus, on the other hand, correlates much more to
human judgment, but it may be expensive to estimate. Thus, our second metric is an external
word embeddings topic coherence metric, which we compute by adopting a strategy similar to that
described in Ding et al. (2018). First, we compute
the average pairwise cosine similarity of the word
embeddings of the top-10 words in a topic using (Mikolov et al., 2013) embeddings. Then, we
compute the overall average of those values for all
the topics (α).

3. **rank-
biased overlap (RBO) (Webber et al., 2010)**

To evaluate how diverse the topics
generated by a single model are, we use the rank-
biased overlap (RBO) (Webber et al., 2010). RBO
compares two topics of the same model. The key
qualities of this measure are twofold: it allows
disjointedness between the lists of topics (i.e., two
topics can have different words in them) and it is
weighted on the ranking (i.e., two lists that share
some of the same words, albeit at different rankings,
are penalized less than two lists that share the same
words at the highest ranks). We deﬁne ρ as the rank-
biased overlap diversity, that we interpret as the
reciprocal of the standard RBO. ρ is 0 for identical
topics and 1 for completely different topics. Both
metrics are computed on the top-k ranked lists.
Following the state-of-the-art, we consider k = 10.

In [4]:
# global variable used throughout te notebook to update preprocessing steps
UPDATE = False
DIR_REPORTS = 'C:/Users/Pieter-Jan/Documents/Work/Candriam/nlp/ESG/reports/Sectors/'

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Download reports and perform some text processing

In [5]:
df = pd.read_excel("data//Industry_CSR_Datasets.xlsx").dropna().reset_index(drop=True)
df.head()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\Pieter-Jan\Anaconda3\envs\marketDb\lib\site-packages\xlrd\xlsx.py:266: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\Pieter-Jan\Anaconda3\envs\marketDb\lib\site-packages\xlrd\xlsx.py:312: DeprecationWarning: This method will be removed in future versions.  Use 'tree.iter()' or 'list(tree.iter())' instead.
  for elem in self.tree.iter() if Element_has_iter else self.tree.getiterator():
C:\Users\Pieter-Jan\Anaconda3\envs\marketDb\lib\site-packag

,company,industry,sector,year,url
0,Brookfield Property Partners LP,Real Estate Development,Financial,2019.0,https://www.brookfield.com/sites/default/files...
1,PrairieSky Royalty Ltd,Real Estate Development,Financial,2017.0,https://www.prairiesky.com/wp-content/uploads/...
2,PrairieSky Royalty Ltd,Real Estate Development,Financial,2018.0,https://www.prairiesky.com/wp-content/uploads/...
3,PrairieSky Royalty Ltd,Real Estate Development,Financial,2019.0,https://www.prairiesky.com/wp-content/uploads/...
4,Summit Hotel Properties Inc.,Real Estate Development,Financial,2018.0,https://www.responsibilityreports.com/HostedDa...


In [12]:
# download reports and store
df_download, could_not_download = preprocess.download_reports(df=df, directory=DIR_REPORTS, update=True)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 2/2 [00:03<00:00,  1.58s/it]


In [15]:
df_download.shape

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(168, 6)

In [22]:
# onlyfiles = [f for f in os.listdir(DIR_REPORTS) if os.path.isfile(os.path.join(DIR_REPORTS, f))]
# df = df[~df['company'].isin(could_not_download)]
# df["filename"] = onlyfiles

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
df_processed = preprocess.load_processed_text(
    df_download,
    dir_read_pdf=DIR_REPORTS, 
    columns_to_keep = ["company","industry", "sector", "year", "url", "filename"],
    file_processed_text="output/CRS_processed_pdfMiner.txt",
    n_min_word_paragraph=50, 
    n_max_word_paragraph=125,  
    update=True,
    method_extract_content = "pdfMiner"
    )

'Text']}, streams=[<PDFStream(1059): len=554, {'BBox': [0.0, 792.0, 1224.0, 0.0], 'Filter': /'FlateDecode', 'Length': 315, 'Matrix': [1.0, 0.0, 0.0, 1.0, 0.0, 0.0], 'Resources': {'ColorSpace': {'CS0': <PDFObjRef:2419>}, 'ExtGState': {'GS0': <PDFObjRef:2418>}, 'Font': {'T1_0': <PDFObjRef:1060>, 'T1_1': <PDFObjRef:1097>}, 'ProcSet': [/'PDF', /'Text']}, 'Subtype': /'Form'}>], ctm=(1.0, 0.0, 0.0, 1.0, 0.0, 0.0)
2020-11-18 18:32:24.514 INFO    pdfminer.pdfinterp: Processing xobj: <PDFStream(1015): raw=36664, {'BitsPerComponent': 8, 'ColorSpace': <PDFObjRef:2419>, 'Filter': /'DCTDecode', 'Height': 318, 'Intent': /'RelativeColorimetric', 'Length': 36662, 'Metadata': <PDFObjRef:1012>, 'Name': /'X', 'SMask': <PDFObjRef:1014>, 'Subtype': /'Image', 'Type': /'XObject', 'Width': 485}>
2020-11-18 18:32:24.533 INFO    pdfminer.pdfpage: Page: {'Annots': <PDFObjRef:1018>, 'ArtBox': [0.0, 0.0, 1224.0, 792.0], 'BleedBox': [0.0, 0.0, 1224.0, 792.0], 'Contents': <PDFObjRef:1019>, 'CropBox': [0.0, 0.0, 1224

In [9]:
print(df_processed.shape)
df_processed.head()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
(6283, 7)


,company,industry,sector,year,url,filename,paragraph
0,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,ArmadaHofflerPropertiesInc-2019.pdf,Armada Hoffler Throughout Armada Hofflers yea...
1,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,ArmadaHofflerPropertiesInc-2019.pdf,We recognize the importance of sustainability ...
2,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,ArmadaHofflerPropertiesInc-2019.pdf,As we explore new projects we approach the pla...
3,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,ArmadaHofflerPropertiesInc-2019.pdf,As forty year old company our focus has always...
4,Brookfield Property Partners LP,Real Estate Development,Financial,2019,https://www.brookfield.com/sites/default/files...,ArmadaHofflerPropertiesInc-2019.pdf,employer match Employer paid life and AD insur...


In [10]:
# save only the paragraph to a text file
df_processed["paragraph"].to_csv('output/CSR_processed_raw_pdfMiner.txt', sep='\t', index=False, header=False)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# filter out company names
own_stop_words = ["prairiesky", "cousin", "use", "uk", "france", "landlord", "abs", "london"]
fsi_stop_words = df_processed["company"].unique().tolist() 

# our list contains all english stop words + companies names + specific keywords
stop_words = text.ENGLISH_STOP_WORDS.union(fsi_stop_words)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# load spacy model to lematize text
nlp = preprocess.load_spacy_model()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [13]:
# lematize text and remove stopwords
# use method 1

df_processed = preprocess.load_lemmatize(
    data=df_processed, 
    dir_file='output/CSR_processed_cleaned_pdfMiner.txt', 
    stop_words=stop_words, 
    nlp=nlp, 
    method=1, 
    update=True
)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
100%|██████████| 6283/6283 [02:02<00:00, 51.49it/s]


## 1. Contextualized Topic Modelling

## 1.1 Prepare data for model

In [14]:
with open('output/CSR_processed_cleaned_pdfMiner.txt',"r") as fr:
    text_cleaned = [doc.split() for doc in fr.read().splitlines()] # load text for NPMI

dictionary = Dictionary(text_cleaned)

'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''

dictionary.filter_extremes(no_below=10, no_above=0.10, keep_n= 100000)
print(len(dictionary.token2id))

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 18:34:51.979 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary(0 unique tokens: [])
2020-11-18 18:34:53.003 INFO    gensim.corpora.dictionary: built Dictionary(8372 unique tokens: ['armada', 'central', 'citizenship', 'come', 'corporate']...) from 6283 documents (total 331204 corpus positions)
2020-11-18 18:34:53.028 INFO    gensim.corpora.dictionary: discarding 5992 tokens: [('corporate', 870), ('environmental', 915), ('good', 656), ('hofflers', 1), ('intentional', 1), ('paramount', 5), ('profitability', 3), ('tenet', 1), ('thoughtfully', 1), ('year', 1327)]...
2020-11-18 18:34:53.03

In [15]:
bow_corpus = [dictionary.doc2bow(doc) for doc in text_cleaned]

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
tf_array = matutils.corpus2dense(bow_corpus, num_terms=len(dictionary.token2id)).T
tf_array.shape

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(6283, 2380)

In [17]:
# convert to sparse matrix
tf_array_sparse = sparse.csr_matrix(tf_array)
tf_array_sparse

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<6283x2380 sparse matrix of type '<class 'numpy.float32'>'
	with 166814 stored elements in Compressed Sparse Row format>

In [19]:
# create or load bert embeddings (either use raw text or clean text)
# we can expirment with both
embeddings_bert = preprocess.load_bert_embeddings(
        text_dir="output/CSR_processed_raw_pdfMiner.txt", 
        model="distiluse-base-multilingual-cased",
        dir_embeddings="output/CRS_bertEmbeddings_pdfMiner.npy",
        update=True
)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 18:52:34.793 INFO    root: Load pretrained SentenceTransformer: distiluse-base-multilingual-cased
2020-11-18 18:52:34.795 INFO    root: Did not find folder distiluse-base-multilingual-cased. Assume to download model from server.
2020-11-18 18:52:34.817 INFO    root: Load SentenceTransformer from folder: C:\Users\Pieter-Jan/.cache\torch\sentence_transformers\sbert.net_models_distiluse-base-multilingual-cased
2020-11-18 18:52:40.308 INFO    root: Use pytorch device: cpu
Batches: 100%|██████████| 32/32 [24:50<00:00, 46.56s/it]


In [20]:
embeddings_bert.shape

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(6283, 512)

In [21]:
# ivert dictionary
inv_token2id = {v: k for k, v in dictionary.token2id.items()}
# create dataset
training_dataset = CTMDataset(tf_array_sparse, embeddings_bert, inv_token2id)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 1.2 Train model

In [22]:
random.seed(69)
ctm = CTM(
    input_size=len(dictionary.token2id), 
    bert_input_size=512, 
    n_components=20, 
    inference_type="combined", 
    num_epochs=10,
    reduce_on_plateau=True
    )
ctm.fit(training_dataset) # run model

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Settings: 
                   N Components: 20
                   Topic Prior Mean: 0.0
                   Topic Prior Variance: 0.95
                   Model Type: prodLDA
                   Hidden Sizes: (100, 100)
                   Activation: softplus
                   Dropout: 0.2
                   Learn Priors: True
                   Learning Rate: 0.002
                   Momentum: 0.99
                   Reduce On Plateau: True
                   Save Dir: None
Epoch: [1/10]	Samples: [6283/62830]	Train Loss: 291.5454813817419	Time: 0:00:16.991350
Epoch: [2/10]	Samples: [12566/62830]	Train Loss: 277.4

## 1.3 Evaluate topics

In [23]:
cmt_topics_l = ctm.get_topic_lists(10)
cmt_topics_d = {}
for i in range(len(cmt_topics_l)):
    cmt_topics_d[i] = cmt_topics_l[i]

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
pd.DataFrame.from_dict(cmt_topics_d).T

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9
0,training,focus,need,programme,deliver,survey,local,opportunity,staff,wide
1,economic,hall,charter,financial,material,indicator,significant,investment,social,climate
2,senior,disclosure,assurance,organisation,indicator,statement,female,index,real,review
3,recycle,landfill,recycling,disposal,reuse,quantity,tonne,hazardous,fuel,divert
4,prairiesky,royalty,operator,industry,prairieskys,long,share,lease,company,responsibility
5,charlotte,white,collar,factory,brunel,angel,oxford,street,embody,stephen
6,board,director,governance,committee,policy,social,executive,high,responsibility,health
7,local,people,activity,skill,employment,partner,charity,contribution,volunteer,wellbeing
8,park,bullre,data,reading,italie,westquay,sell,birmingham,highcross,brent
9,scope,intensity,like,mtco,derwent,indir,greenhouse,assurance,kgco,location


### 1.3.1 Normalized Point-wise Mutual Information

In [25]:
npmi = CoherenceNPMI(texts=text_cleaned, topics=ctm.get_topic_lists(10))
npmi.score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 19:20:14.768 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary(0 unique tokens: [])
2020-11-18 19:20:15.230 INFO    gensim.corpora.dictionary: built Dictionary(8372 unique tokens: ['armada', 'central', 'citizenship', 'come', 'corporate']...) from 6283 documents (total 331204 corpus positions)
2020-11-18 19:20:15.241 INFO    gensim.topic_coherence.probability_estimation: using ParallelWordOccurrenceAccumulator(processes=3, batch_size=64) to estimate probabilities from sliding windows
2020-11-18 19:20:19.356 INFO    gensim.topic_coherence.text_analysis: 1 batches submitted to accumulat

0.05363407861000221

### 1.3.2 External Word Embeddings Topic Coherence

In [26]:
CoherenceWordEmbeddings(ctm.get_topic_lists(10)).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 19:20:23.795 INFO    gensim.models.utils_any2vec: loading projection weights from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
2020-11-18 19:21:25.006 INFO    gensim.models.utils_any2vec: loaded (3000000, 300) matrix from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


0.13012877

### 1.3.3 Rank-Biased Overlap 

In [27]:
InvertedRBO(ctm.get_topic_lists(10)).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9903040631643233

## 2. Latent Dirichlet Allocation

## 2.1 Train model

In [28]:
num_topics = 20
lda_model =  LdaMulticore(
    corpus=bow_corpus, 
    num_topics = num_topics, 
    id2word = dictionary,                                    
    passes = 10,
    workers = 2
    )

lete" + 0.010*"green" + 0.010*"site" + 0.010*"recycling" + 0.008*"rating" + 0.008*"build" + 0.008*"refurbishment"
2020-11-18 19:22:00.856 INFO    gensim.models.ldamodel: topic #8 (0.050): 0.035*"tonne" + 0.035*"recycle" + 0.030*"landfill" + 0.025*"hazardous" + 0.025*"fuel" + 0.019*"reuse" + 0.019*"material" + 0.019*"send" + 0.019*"food" + 0.019*"assurance"
2020-11-18 19:22:00.857 INFO    gensim.models.ldamodel: topic diff=0.772409, rho=0.350468
2020-11-18 19:22:01.080 INFO    gensim.models.ldamodel: -7.018 per-word bound, 129.6 perplexity estimate based on a held-out corpus of 283 documents with 9843 words
2020-11-18 19:22:01.082 INFO    gensim.models.ldamulticore: PROGRESS: pass 5, dispatched chunk #0 = documents up to #2000/6283, outstanding queue size 1
2020-11-18 19:22:01.100 INFO    gensim.models.ldamulticore: PROGRESS: pass 5, dispatched chunk #1 = documents up to #4000/6283, outstanding queue size 2
2020-11-18 19:22:01.125 INFO    gensim.models.ldamulticore: PROGRESS: pass 5, di

## 2.2 Evaluate topics

In [29]:
lda_topics_d = {}
lda_topics_l = []
for i in range(num_topics):
    t = [w[0] for w in lda_model.show_topic(i)[0:10]]
    lda_topics_d[i+1] = t
    lda_topics_l.append(t)    

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
# show topics
# evert row is a topic
pd.DataFrame.from_dict(lda_topics_d).T

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9
1,scope,like,indicator,intensity,greenhouse,factor,source,indirect,utility,kgco
2,climate,term,supplier,design,supply,progress,chain,long,risk,construction
3,training,staff,volunteer,local,programme,opportunity,female,diversity,hour,skill
4,prairiesky,responsibility,operator,value,royalty,conduct,company,right,environment,shareholder
5,scope,mtco,people,indir,employment,location,market,young,skill,local
6,landlord,electricity,elec,like,fuels,intensity,meter,natural,obtain,renewable
7,cost,data,million,value,charge,investment,note,available,levy,capital
8,visitor,positive,rainwater,service,focus,stakeholder,local,deliver,relationship,landlord
9,recycle,tonne,landfill,hazardous,fuel,material,food,reuse,send,recycling
10,risk,board,governance,committee,climate,director,policy,executive,responsibility,process


### 2.2.1 Normalized Point-wise Mutual Information

In [31]:
npmi = CoherenceNPMI(texts=text_cleaned, topics=lda_topics_l)
npmi.score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 19:22:23.754 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary(0 unique tokens: [])
2020-11-18 19:22:24.457 INFO    gensim.corpora.dictionary: built Dictionary(8372 unique tokens: ['armada', 'central', 'citizenship', 'come', 'corporate']...) from 6283 documents (total 331204 corpus positions)
2020-11-18 19:22:24.470 INFO    gensim.topic_coherence.probability_estimation: using ParallelWordOccurrenceAccumulator(processes=3, batch_size=64) to estimate probabilities from sliding windows
2020-11-18 19:22:28.132 INFO    gensim.topic_coherence.text_analysis: 1 batches submitted to accumulat

0.0574448734166926

### 2.2.2 External Word Embeddings Topic Coherence

In [32]:
CoherenceWordEmbeddings(lda_topics_l).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 19:22:32.887 INFO    gensim.models.utils_any2vec: loading projection weights from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
2020-11-18 19:23:21.754 INFO    gensim.models.utils_any2vec: loaded (3000000, 300) matrix from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


0.13026394

### 2.2.3 Rank-Biased Overlap 

In [33]:
InvertedRBO(lda_topics_l).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9794654136539473

## 3. Bert Topic

## 3.1 Train model

In [34]:
random.seed(69)

with open("output/CSR_processed_cleaned_pdfMiner.txt","r") as fr:
    docs = [doc for doc in fr.read().splitlines()] 

model = BERTopic(verbose=True)
topics = model.fit_transform(docs, embeddings_bert)

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 19:23:46,046 - BERTopic - Reduced dimensionality with UMAP
2020-11-18 19:23:46.046 INFO    BERTopic: Reduced dimensionality with UMAP
2020-11-18 19:23:47,284 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2020-11-18 19:23:47.284 INFO    BERTopic: Clustered UMAP embeddings with HDBSCAN


## 3.2 Evaluate topics

In [35]:
topcis_b = model.get_topics()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
# extract words for each topic
topics_k = {}
for k,v in topcis_b.items():
    t_words = []
    for w in v:
        t_words.append(w[0])
    # append the first 10 words
    topics_k[k] = t_words[0:10]

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
topics_bert = list(topics_k.values())
pd.DataFrame.from_dict(topics_k).T

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9
-1,property,environmental,manage,sustainability,performance,development,management,datum,project,risk
0,park,retail,sell,brent,excellent,shopping,street,saint,centre,london
1,prairiesky,operator,royalty,land,lease,mineral,canada,liability,industry,reclamation
2,page,organisation,annual,governance,account,statement,general,stakeholder,body,topic
3,mzftifbludn,hyiow,gebiawvpsy,jwlm,vitwgjgayqvxze,corporate,program,hwjixe,norefer,realty
4,assurance,matter,subject,procedure,perform,accordance,independence,conclusion,criterion,derwent
5,employee,number,female,permanent,gender,male,contract,record,senior,leave
6,oxford,street,buckley,embody,copyright,site,brunel,build,pentonville,charlotte
7,landlord,emission,biomass,scope,electricity,like,building,total,tenant,intensity
8,tonne,recovery,water,building,street,incineration,angel,like,exclude,total


### 3.2.1 Normalized Point-wise Mutual Information

In [38]:
npmi = CoherenceNPMI(texts=text_cleaned, topics=topics_bert)
npmi.score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 19:23:49.220 INFO    gensim.corpora.dictionary: adding document #0 to Dictionary(0 unique tokens: [])
2020-11-18 19:23:49.596 INFO    gensim.corpora.dictionary: built Dictionary(8372 unique tokens: ['armada', 'central', 'citizenship', 'come', 'corporate']...) from 6283 documents (total 331204 corpus positions)
2020-11-18 19:23:49.601 INFO    gensim.topic_coherence.probability_estimation: using ParallelWordOccurrenceAccumulator(processes=3, batch_size=64) to estimate probabilities from sliding windows
2020-11-18 19:23:52.711 INFO    gensim.topic_coherence.text_analysis: 1 batches submitted to accumulat

0.1746779890206453

### 3.2.2 External Word Embeddings Topic Coherence

In [39]:
CoherenceWordEmbeddings(topics_bert).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-18 19:23:58.692 INFO    gensim.models.utils_any2vec: loading projection weights from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz
2020-11-18 19:24:48.158 INFO    gensim.models.utils_any2vec: loaded (3000000, 300) matrix from C:\Users\Pieter-Jan/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


0.1362753

### 3.2.3 Rank-Biased Overlap 

In [40]:
InvertedRBO(topics_bert).score()

C:\Users\Pieter-Jan\AppData\Roaming\Python\Python38\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0.9891851583628571